In [81]:
import pandas as pd
import json

# 데이터 불러오기 및 전처리
df1 = pd.read_csv('data/교통사고 데이터/전국교통사고다발지역(2012-2021).csv', encoding='euc-kr')
# 2013-2016년 데이터만 추출
df1 = df1[(df1['사고연도'] >= 2014) & (df1['사고연도'] <= 2016)]

drop_list1 = ['사고연도', '사고유형구분', '데이터기준일자', '제공기관코드', '제공기관명']
df1 = df1.drop(columns=drop_list1)

df2 = pd.read_csv('data/교통사고 데이터/전국교통사고다발지역(2017-2023).csv', encoding='euc-kr')
drop_list2 = ['법정동코드']
df2 = df2.drop(columns=drop_list2)

df = pd.concat([df1, df2])

# '위치코드'의 앞 5자리 숫자로 그룹화하고 사고건수를 합산
df['위치코드_시군구'] = df['위치코드'].astype(str).str[:5]

# 그룹화하고 첫 번째 '사고지역위치명'의 처음 두 단어를 사용
df_grouped = df.groupby('위치코드_시군구', as_index=False).agg({
    '사고건수': 'sum',
    '사고지역위치명': lambda x: ' '.join(x.iloc[0].split()[:2])  # 첫 번째 위치명에서 처음 두 단어 추출
})
df_grouped = df_grouped.rename(columns={'사고지역위치명': '시군구명'})

# 결과를 JSON 파일로 저장
df_grouped.to_json('data/accident_data.json', orient='records', force_ascii=False)
print(df)

      사고지역관리번호      위치코드   사고다발지역시도시군구                    사고지역위치명  사고건수  사상자수  \
1468   2015046  43130003     충청북도 충주시3      충청북도 충주시 교현동(탑이용원 부근)     4     5   
1574   2015046  47111001  경상북도 포항시 남구1   경상북도 포항시 남구 해도동(경아슈퍼 부근)     5     5   
1695   2015046  47210003     경상북도 영주시3     경상북도 영주시 휴천동(영남커피숍 부근)     4     4   
1711   2015046  47210002     경상북도 영주시2      경상북도 영주시 가흥동(정휘상회 부근)     4     4   
1835   2015046  47210001     경상북도 영주시1  경상북도 영주시 영주동(영주신용협동조합 부근)     8     8   
...        ...       ...           ...                        ...   ...   ...   
4629   2024056  52790002       전북 고창군2        전북 고창군 고창읍(바지라기 부근)     8     9   
4630   2024056  52790003       전북 고창군3       전북 고창군 흥덕면(제하사거리 부근)     4     5   
4631   2024056  52800001       전북 부안군1       전북 부안군 부안읍(아담사거리 부근)    12    13   
4632   2024056  52800002       전북 부안군2      전북 부안군 부안읍(터미널사거리 부근)    11    11   
4633   2024056  52800003       전북 부안군3       전북 부안군 부안읍(송학사거리 부근)     8     9   

      사망자수  중상자수  경상자수  부상신

In [82]:
df_grouped

,위치코드_시군구,사고건수,시군구명
0,11041,25,서울특별시 중구
1,11051,12,서울특별시 중구
2,11061,9,서울특별시 종로구
3,11071,64,서울특별시 종로구
4,11081,32,서울특별시 서대문구
...,...,...,...
442,52740,6,전북 장수군
443,52750,6,전북 임실군
444,52770,16,전북 순창군
445,52790,21,전북 고창군


In [79]:
df_sorted = df_grouped.sort_values('사고건수', ascending=False)

# 결과를 JSON 파일로 저장 (표2 표시용)
df_sorted.head(10).to_json('data/top10_accident_grouped.json', orient='records', force_ascii=False)

# 결과 출력
print(df_sorted)

    위치코드_시군구  사고건수          시군구명
46     11560  1601    서울특별시 영등포구
51     11710  1517     서울특별시 송파구
188    27290  1412     대구광역시 달서구
22     11230  1399    서울특별시 동대문구
216    30170  1393      대전광역시 서구
..       ...   ...           ...
128    20234     4      경상남도 창원시
67     13074     4       경기도 파주시
66     13064     4      경기도 의정부시
207    28720     4     인천광역시 옹진군
133    21035     3  제주특별자치도 서귀포시

[447 rows x 3 columns]


In [80]:
# '사고지역위치명'이 같은 것들끼리 사고건수를 합산
df_grouped2 = df.groupby('사고지역위치명', as_index=False).agg({
    '사고건수': 'sum'
})
df_sorted2 = df_grouped2.sort_values('사고건수', ascending=False)

# 결과를 JSON 파일로 저장 (표1 표시용)
df_sorted2.head(10).to_json('data/top10_accident_locations.json', orient='records', force_ascii=False)

# 결과 출력
print(df_sorted2)

                           사고지역위치명  사고건수
3965       부산광역시 연제구 연산동(연산교차로 인근)   382
873   경기도 성남시 중원구 하대원동(모란시장사거리 인근)   361
963      경기도 수원시 팔달구 인계동(나혜석거리 인근)   303
3233       대구광역시 수성구 지산동(황금네거리 인근)   293
859      경기도 성남시 중원구 성남동(모란삼거리 인근)   293
...                            ...   ...
2598        광주광역시 광산구 월곡동(대반초교 부근)     1
3586        부산광역시 금정구 구서동(두실초교 부근)     1
4383        서울특별시 관악구 봉천동(원당초교 부근)     1
3570       부산광역시 강서구 대저1동(대상초교 부근)     1
6082      인천광역시 연수구 옥련동(인천송도초교 부근)     1

[7652 rows x 2 columns]


In [51]:
# 법정구역 GeoJSON 파일에서 시군구코드와 위치코드 매핑 정보 불러오기
with open('data/법정구역_시군구.geojson', 'r', encoding='utf-8') as f:
    geojson = json.load(f)

# GeoJSON에서 시군구코드와 위치코드 추출
location_mapping = {}
for feature in geojson['features']:
    code = feature['properties']['SIG_CD']
    name = feature['properties']['SIG_KOR_NM']
    location_mapping[name] = code
location_mapping

{'종로구': '11110',
 '중구': '31110',
 '용산구': '11170',
 '성동구': '11200',
 '광진구': '11215',
 '동대문구': '11230',
 '중랑구': '11260',
 '성북구': '11290',
 '강북구': '11305',
 '도봉구': '11320',
 '노원구': '11350',
 '은평구': '11380',
 '서대문구': '11410',
 '마포구': '11440',
 '양천구': '11470',
 '강서구': '26440',
 '구로구': '11530',
 '금천구': '11545',
 '영등포구': '11560',
 '동작구': '11590',
 '관악구': '11620',
 '서초구': '11650',
 '강남구': '11680',
 '송파구': '11710',
 '강동구': '11740',
 '서구': '30170',
 '동구': '31170',
 '영도구': '26200',
 '부산진구': '26230',
 '동래구': '26260',
 '남구': '31140',
 '북구': '31200',
 '해운대구': '26350',
 '사하구': '26380',
 '금정구': '26410',
 '연제구': '26470',
 '수영구': '26500',
 '사상구': '26530',
 '기장군': '26710',
 '수성구': '27260',
 '달서구': '27290',
 '달성군': '27710',
 '군위군': '27720',
 '미추홀구': '28177',
 '연수구': '28185',
 '남동구': '28200',
 '부평구': '28237',
 '계양구': '28245',
 '강화군': '28710',
 '옹진군': '28720',
 '광산구': '29200',
 '유성구': '30200',
 '대덕구': '30230',
 '울주군': '31710',
 '세종특별자치시': '36110',
 '수원시 장안구': '41111',
 '수원시 권선구': '41113',
 '수원시 팔달구': '41115